In [1]:
import ImageLoader
import ImageCompare
import Database

File already exists:
	c:\users\jaden\dropbox\cog\dlibmodels\dlib_models\dlib_face_recognition_resnet_model_v1.dat
File already exists:
	c:\users\jaden\dropbox\cog\dlibmodels\dlib_models\shape_predictor_68_face_landmarks.dat


ModuleNotFoundError: No module named 'ImageCompare'

In [ ]:
def identify():
    """
    Takes a picture with configured camera and identifies all of the faces in the picture
    Parameters:
    -----------
    None
    
    Returns:
    --------
    names (list)
        the list of the name of each person in the picture
    """
    descs = ImageLoader.describe()
    ImageCompare.compare_faceA
            